In [1]:
#!/usr/bin/env python
# coding: utf-8

import os,sys
import numpy as np
import math
import scipy
from scipy import ndimage
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
#from PIL import Image
import random
import keras
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.models import Sequential
import keras.backend as K
import time

D:\toolkits.win\Anaconda3-5.3.1\envs\tensorflow-gpu-1.10\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\toolkits.win\Anaconda3-5.3.1\envs\tensorflow-gpu-1.10\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\toolkits.win\Anaconda3-5.3.1\envs\tensorflow-gpu-1.10\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\toolkit

In [2]:
print(K.image_data_format())

channels_last


In [3]:
def file2array(path, delimiter):
    recordlist = []
    fp = open(path, 'r', encoding='utf-8')
    content = fp.read()     # content现在是一行字符串，该字符串包含文件所有内容
    fp.close()
    rowlist = content.splitlines()  # 按行转换为一维表，splitlines默认参数是‘\n’
    # 逐行遍历
    # 结果按分隔符分割为行向量
    recordlist = [row.split(delimiter) for row in rowlist if row.strip()]
    return np.array(recordlist)

In [4]:
#import numpy as np
import scipy.ndimage


def ndarray_zoom_scaling(label, new_h, new_w):
    """
    Implement scaling for ndarray with scipy.ndimage.zoom
    :param label: [H, W] or [H, W, C]
    :return: label_new: [new_h, new_w] or [new_h, new_w, C]
    Examples
    --------
    ori_arr = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]], dtype=np.int32)
    new_arr = ndarray_zoom_scaling(ori_arr, new_h=8, new_w=10)
    >> print(new_arr)
    [[1 1 1 2 2 2 2 3 3 3]
     [1 1 1 2 2 2 2 3 3 3]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [7 7 7 8 8 8 8 9 9 9]
     [7 7 7 8 8 8 8 9 9 9]]
    """
    scale_h = new_h / label.shape[0]
    scale_w = new_w / label.shape[1]
    if len(label.shape) == 2:
        label_new = scipy.ndimage.zoom(label, zoom=[scale_h, scale_w], order=0)
    else:
        label_new = scipy.ndimage.zoom(label, zoom=[scale_h, scale_w, 1], order=0)
    return label_new


def ndarray_nearest_neighbour_scaling(label, new_h, new_w):
    """
    Implement nearest neighbour scaling for ndarray
    :param label: [H, W] or [H, W, C]
    :return: label_new: [new_h, new_w] or [new_h, new_w, C]
    Examples
    --------
    ori_arr = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]], dtype=np.int32)
    new_arr = ndarray_nearest_neighbour_scaling(ori_arr, new_h=8, new_w=10)
    >> print(new_arr)
    [[1 1 1 1 2 2 2 3 3 3]
     [1 1 1 1 2 2 2 3 3 3]
     [1 1 1 1 2 2 2 3 3 3]
     [4 4 4 4 5 5 5 6 6 6]
     [4 4 4 4 5 5 5 6 6 6]
     [4 4 4 4 5 5 5 6 6 6]
     [7 7 7 7 8 8 8 9 9 9]
     [7 7 7 7 8 8 8 9 9 9]]
    """
#    if len(label.shape) == 2:
#        label_new = np.zeros([new_h, new_w], dtype=label.dtype)
#    else:
#        label_new = np.zeros([new_h, new_w, label.shape[2]], dtype=label.dtype)
    label_new = np.zeros([new_h, new_w, label.shape[2]], dtype=label.dtype)

    scale_h = new_h / label.shape[0]
    scale_w = new_w / label.shape[1]

    y_pos = np.arange(new_h)
    x_pos = np.arange(new_w)
    y_pos = np.floor(y_pos / scale_h).astype(np.int32)
    x_pos = np.floor(x_pos / scale_w).astype(np.int32)

    y_pos = y_pos.reshape(y_pos.shape[0], 1)
    y_pos = np.tile(y_pos, (1, new_w))
    x_pos = np.tile(x_pos, (new_h, 1))
    assert y_pos.shape == x_pos.shape

    label_new[:, :] = label[y_pos[:, :], x_pos[:, :]]
    return label_new


In [5]:
def DataSet(iFold,noise_tag,dup_tag,rand_tag):
    #
    train_path =("../Data_CNN_Indus/F%d/SECOM_samples_train_F%d") % (iFold, iFold)    
    test_path  =("../Data_CNN_Indus/F%d/SECOM_samples_test_F%d") % (iFold, iFold)
    #
    data_train = file2array(train_path, " ")
    nrow_train = data_train.shape[0]
    ncol_train = data_train.shape[1]
    #
    data_test  = file2array(test_path, " ")
    nrow_test  = data_test.shape[0]
    ncol_test  = data_test.shape[1]
    #
    if ncol_train != ncol_test:
        print('Numbers of features are not consistant.')
    if nrow_train + nrow_test != 1567:
        print('Numbers of samples are not consistant.')
    #
    train_path =("../Data_CNN_Indus/F%d/SECOM_labels_train_F%d") % (iFold, iFold)    
    test_path  =("../Data_CNN_Indus/F%d/SECOM_labels_test_F%d") % (iFold, iFold)
    #
    lab_train = file2array(train_path, " ")
    #
    lab_test  = file2array(test_path, " ")
    #
    num_feature = ncol_train
    num_class = 2
    
    tmp_cnt_per_class = np.zeros(num_class)
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        tmp_cnt_per_class[(int)(tmp)] = tmp_cnt_per_class[(int)(tmp)] + 1
        
    tmp_max = tmp_cnt_per_class[0]
    tmp_ind = 0
    for index in range(num_class):
        if tmp_cnt_per_class[index] > tmp_max:
            tmp_max = tmp_cnt_per_class[index]
            tmp_ind = index
            
    num_dup = np.zeros(num_class)
    for index in range(num_class):
        if (int)(dup_tag):
            num_dup[index] = math.floor(tmp_cnt_per_class[tmp_ind]/tmp_cnt_per_class[index])
        else:
            num_dup[index] = 1
        
    nrow_train_dup = 0
    for index in range(num_class):
        nrow_train_dup = nrow_train_dup + tmp_cnt_per_class[index] * num_dup[index]
        
    nrow_train_dup = (int)(nrow_train_dup)
    
    X_train = np.empty((nrow_train_dup, num_feature), dtype=float)
    Y_train = np.empty((nrow_train_dup, num_class), dtype=float)

    sam_count = 0
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        nDup = num_dup[(int)(tmp)]
        nDup = (int)(nDup)
        for iSam in range(nDup):
            tmp_cnt = 0
            for i in range(num_feature):
                X_train[sam_count,i] = float(data_train[index,i])
            sam_count = sam_count + 1

    sam_count = 0
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        nDup = num_dup[(int)(tmp)]
        nDup = (int)(nDup)
        for iSam in range(nDup):
            tmp_v = np.zeros(num_class)
            tmp_v[(int)(tmp)] = 1
            Y_train[sam_count] = tmp_v
            sam_count = sam_count + 1
    
    X_test  = np.empty((nrow_test, num_feature), dtype=float)
    Y_test  = np.empty((nrow_test, num_class), dtype=float)

    for index in range(nrow_test):
        tmp_cnt = 0
        for i in range(num_feature):
            X_test[index,i] = float(data_test[index,i])
    
    for index in range(nrow_test):
        tmp = lab_test[index,0]
        tmp_v = np.zeros(num_class)
        tmp_v[(int)(tmp)] = 1
        Y_test[index] = tmp_v
        
    if (int)(noise_tag):
        noise = np.random.normal(loc=0,scale=1e-2,size=X_train.shape)
        output = X_train + noise
        X_train = np.concatenate((X_train, output), axis=0)
        Y_train = np.concatenate((Y_train, Y_train), axis=0)
    
    index = [i for i in range(len(X_train))]
    random.shuffle(index)
    X_train = X_train[index]
    Y_train = Y_train[index]
    
    index = [i for i in range(len(X_test))]
    random.shuffle(index)
    X_test = X_test[index]    
    Y_test = Y_test[index]
    
    #X_train_new = np.empty((nrow_train_dup, 224, 224, 3))
    #Y_train_new = Y_train
    
    #for index in range(nrow_train_dup):
    #    tmp_new = ndarray_nearest_neighbour_scaling(X_train[index],224,224)
    #    for c in range(3):
    #        X_train_new[index,:,:,c] = tmp_new[:,:,0]
    
    #X_test_new = np.empty((nrow_test, 224, 224, 3))
    #Y_test_new = Y_test
    
    #for index in range(nrow_test):
    #    tmp_new = ndarray_nearest_neighbour_scaling(X_test[index],224,224)
    #    for c in range(3):
    #        X_test_new[index,:,:,c] = tmp_new[:,:,0]
            
    #del X_train, X_test
    #gc.collect()

    return X_train,Y_train,X_test,Y_test

In [6]:
#import tensorflow as tf
def LeakyRelu(x, leak=0.01, name="LeakyRelu"):
   with tf.variable_scope(name):
     f1 = 0.5 * (1 + leak)
     f2 = 0.5 * (1 - leak)
     return f1 * x + f2 * tf.abs(x)

In [7]:
def neural(channel,length,classes):
    input_shape = (channel,length)
    if K.image_data_format() == "channels_last":#确认输入维度,就是channel是在开头，还是结尾
        input_shape = (length,channel)
    model = Sequential()#顺序模型（keras中包括顺序模型和函数式API两种方式）
    #model.add(Conv2D(6,(3,3),padding="valid",activation=LeakyRelu,input_shape=input_shape,name="conv1"))
    #model.add(Conv2D(6,(3,3),padding="valid",input_shape=input_shape,name="conv1"))
    #model.add(LeakyReLU(alpha=0.01))
    #model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),name="pool1"))
    #model.add(Conv2D(12,(3,3),padding="valid",activation=LeakyRelu,name="conv2",))
    #model.add(Conv2D(12,(3,3),padding="valid",name="conv2",))
    #model.add(LeakyReLU(alpha=0.01))
    #model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),name="pool2"))
    #model.add(Flatten())
    #model.add(Dense(192,activation="relu",name="fc1"))
    model.add(Dense(600,activation=LeakyRelu,input_dim=length,name="fc1"))
    #model.add(Dense(600,activation=LeakyRelu,name="fc2"))
    #model.add(Dense(768,activation=LeakyRelu,name="fc3"))
    model.add(Dense(192,activation=LeakyRelu,name="fc4"))
    model.add(Dense(classes,activation="softmax",name="fc5"))
    return model


In [8]:
def get_statistics(model,sam_data,lab_data):
    num_class = 2

    #
    test_rows = len(lab_data)
    test_prc = np.zeros(num_class)
    test_rec = np.zeros(num_class)
    test_Fva = np.zeros(num_class)
    test_BER = np.zeros(num_class)
    tmp_beta = 1
    yhat_label = model.predict_classes(sam_data, batch_size=32, verbose=1)
    true_labels = np.zeros(len(lab_data))
    for index in range(len(lab_data)):
        tmp_vec = lab_data[index]
        tmp_max = tmp_vec[0]
        tmp_ind = 0
        for i in range(num_class):
            if tmp_max < tmp_vec[i]:
                tmp_max = tmp_vec[i]
                tmp_ind = i
        true_labels[index] = tmp_ind

    test_N_TP = np.zeros(num_class)
    test_N_FP = np.zeros(num_class)
    test_N_TN = np.zeros(num_class)
    test_N_FN = np.zeros(num_class)
    tmp_count = np.zeros(num_class)
    for i in range(test_rows):
        cur_label = yhat_label[i]
        true_label = true_labels[i]
        for j in range(num_class):
            if j == cur_label:
                if j == true_label:
                    test_N_TP[j]=test_N_TP[j]+1
                    tmp_count[j] = tmp_count[j] + 1
                else:
                    test_N_FP[j]=test_N_FP[j]+1
            else:
                if j == true_label:
                    test_N_FN[j]=test_N_FN[j]+1
                    tmp_count[j] = tmp_count[j] + 1
                else:
                    test_N_TN[j]=test_N_TN[j]+1

    for i in range(num_class):
        if test_N_TP[i] > 0:
            test_prc[i] = test_N_TP[i] / (test_N_TP[i] + test_N_FP[i])
        else:
            test_prc[i] = 0

        test_rec[i]= test_N_TP[i] / (test_N_TP[i] + test_N_FN[i])
        test_Fva[i]=(1 + tmp_beta*tmp_beta)*test_rec[i]*test_prc[i]/\
                      (tmp_beta*tmp_beta*(test_rec[i]+test_prc[i]))
        test_BER[i]= test_N_FN[i] / (test_N_TP[i] + test_N_FN[i])
        
    print(tmp_count)
        
    return test_N_TP,test_N_FP,test_N_TN,test_N_FN,tmp_count
    

In [9]:
all_TP_train=[]
all_FP_train=[]
all_TN_train=[]
all_FN_train=[]
all_count_train=[]
all_TP_test=[]
all_FP_test=[]
all_TN_test=[]
all_FN_test=[]
all_count_test=[]
all_time_train=[]
all_time_predict_train=[]
all_time_predict_test=[]

nRun = 10
side_len = 22
num_class = 2
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for fold in range(nRun):
                iFold = fold + 1
                X_train,Y_train,X_test,Y_test = DataSet(iFold,noise_tag,dup_tag,ran_tag)
                print('X_train shape : ',X_train.shape)
                print('Y_train shape : ',Y_train.shape)
                print('X_test shape : ',X_test.shape)
                print('Y_test shape : ',Y_test.shape)
                #print(X_train[0])
                #print(Y_train[0,1])
                print(noise_tag,dup_tag,ran_tag,iFold)
                
                num_feature = X_train.shape[1]

                # # model
                #model = ResNet50(
                #    weights=None,
                #    classes=2
                #)

                ##
                model = neural(1,num_feature,num_class)

                model.compile(optimizer=tf.train.AdamOptimizer(0.001),
                              loss='categorical_crossentropy',
                              metrics=['accuracy'])

                # # train
                start = time.clock()
                model.fit(X_train, Y_train, epochs=100, batch_size=16)
                all_time_train.append(time.clock()-start)

                # # evaluate
                #model.evaluate(X_test, Y_test, batch_size=32)

                # #
                #models.append(model)

                # # save
                #save_path =("./my_resnet_model.h5_F%d") % (iFold)
                #model.save(save_path)

                #
                X_train,Y_train,X_test,Y_test = DataSet(iFold,0,0,ran_tag)
                start = time.clock()
                train_N_TP,train_N_FP,train_N_TN,train_N_FN,train_count = get_statistics(model,X_train,Y_train)
                all_time_predict_train.append(time.clock()-start)
                start = time.clock()
                test_N_TP,test_N_FP,test_N_TN,test_N_FN,test_count = get_statistics(model,X_test,Y_test)
                all_time_predict_test.append(time.clock()-start)
                all_TP_train.append(train_N_TP)
                all_FP_train.append(train_N_FP)
                all_TN_train.append(train_N_TN)
                all_FN_train.append(train_N_FN)
                all_count_train.append(train_count)
                all_TP_test.append(test_N_TP)
                all_FP_test.append(test_N_FP)
                all_TN_test.append(test_N_TN)
                all_FN_test.append(test_N_FN)
                all_count_test.append(test_count)


X_train shape :  (1410, 474)
Y_train shape :  (1410, 2)
X_test shape :  (157, 474)
Y_test shape :  (157, 2)
0 0 0 1
Epoch 1/100
1410/1410 [==============================] - 3s 2ms/step - loss: 0.2888 - acc: 0.9241
Epoch 2/100
1410/1410 [==============================] - 0s 354us/step - loss: 0.2543 - acc: 0.9333
Epoch 3/100
1410/1410 [==============================] - 0s 341us/step - loss: 0.2370 - acc: 0.9333
Epoch 4/100
1410/1410 [==============================] - 0s 341us/step - loss: 0.2279 - acc: 0.9333
Epoch 5/100
1410/1410 [==============================] - 0s 334us/step - loss: 0.2446 - acc: 0.9326
Epoch 6/100
1410/1410 [==============================] - 0s 345us/step - loss: 0.2381 - acc: 0.9319
Epoch 7/100
1410/1410 [==============================] - 0s 333us/step - loss: 0.2229 - acc: 0.9305
Epoch 8/100
1410/1410 [==============================] - 0s 352us/step - loss: 0.2185 - acc: 0.9333
Epoch 9/100
1410/1410 [==============================] - 0s 354us/step - loss: 0.1994 

1410/1410 [==============================] - 0s 335us/step - loss: 0.0280 - acc: 0.9915
Epoch 81/100
1410/1410 [==============================] - 0s 329us/step - loss: 0.0367 - acc: 0.9872
Epoch 82/100
1410/1410 [==============================] - 0s 345us/step - loss: 0.0290 - acc: 0.9915
Epoch 83/100
1410/1410 [==============================] - 0s 351us/step - loss: 0.0350 - acc: 0.9858
Epoch 84/100
1410/1410 [==============================] - 0s 336us/step - loss: 0.0142 - acc: 0.9965
Epoch 85/100
1410/1410 [==============================] - 0s 335us/step - loss: 0.0230 - acc: 0.9915
Epoch 86/100
1410/1410 [==============================] - 0s 325us/step - loss: 0.0354 - acc: 0.9865
Epoch 87/100
1410/1410 [==============================] - 0s 327us/step - loss: 0.0135 - acc: 0.9957
Epoch 88/100
1410/1410 [==============================] - 0s 319us/step - loss: 0.0057 - acc: 0.9986
Epoch 89/100
1410/1410 [==============================] - 0s 348us/step - loss: 0.0041 - acc: 0.9993
Epo

1410/1410 [==============================] - 0s 269us/step - loss: 0.0656 - acc: 0.9766
Epoch 59/100
1410/1410 [==============================] - 0s 259us/step - loss: 0.0685 - acc: 0.9759
Epoch 60/100
1410/1410 [==============================] - 0s 254us/step - loss: 0.1010 - acc: 0.9631
Epoch 61/100
1410/1410 [==============================] - 0s 257us/step - loss: 0.0622 - acc: 0.9801
Epoch 62/100
1410/1410 [==============================] - 0s 277us/step - loss: 0.0536 - acc: 0.9794
Epoch 63/100
1410/1410 [==============================] - 0s 246us/step - loss: 0.0541 - acc: 0.9837
Epoch 64/100
1410/1410 [==============================] - 0s 253us/step - loss: 0.0522 - acc: 0.9787
Epoch 65/100
1410/1410 [==============================] - 0s 267us/step - loss: 0.0518 - acc: 0.9787
Epoch 66/100
1410/1410 [==============================] - 0s 277us/step - loss: 0.0404 - acc: 0.9872
Epoch 67/100
1410/1410 [==============================] - 0s 312us/step - loss: 0.0710 - acc: 0.9738
Epo

D:\toolkits.win\Anaconda3-5.3.1\envs\tensorflow-gpu-1.10\lib\site-packages\ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in double_scalars


X_train shape :  (1410, 474)
Y_train shape :  (1410, 2)
X_test shape :  (157, 474)
Y_test shape :  (157, 2)
0 0 0 3
Epoch 1/100
1410/1410 [==============================] - 1s 647us/step - loss: 0.2620 - acc: 0.9326
Epoch 2/100
1410/1410 [==============================] - 0s 340us/step - loss: 0.2454 - acc: 0.9333
Epoch 3/100
1410/1410 [==============================] - 0s 334us/step - loss: 0.2413 - acc: 0.9333
Epoch 4/100
1410/1410 [==============================] - 0s 355us/step - loss: 0.2364 - acc: 0.9333
Epoch 5/100
1410/1410 [==============================] - 0s 333us/step - loss: 0.2227 - acc: 0.9333
Epoch 6/100
1410/1410 [==============================] - 0s 342us/step - loss: 0.2198 - acc: 0.9326
Epoch 7/100
1410/1410 [==============================] - 0s 333us/step - loss: 0.2114 - acc: 0.9319
Epoch 8/100
1410/1410 [==============================] - 0s 353us/step - loss: 0.2001 - acc: 0.9291
Epoch 9/100
1410/1410 [==============================] - 0s 350us/step - loss: 0.202

...

5238/5238 [==============================] - 2s 398us/step - loss: 8.0129 - acc: 0.5029
Epoch 86/100
5238/5238 [==============================] - 2s 389us/step - loss: 8.0129 - acc: 0.5029
Epoch 87/100
5238/5238 [==============================] - 2s 398us/step - loss: 8.0129 - acc: 0.5029
Epoch 88/100
5238/5238 [==============================] - 2s 400us/step - loss: 8.0129 - acc: 0.5029
Epoch 89/100
5238/5238 [==============================] - 2s 394us/step - loss: 8.0129 - acc: 0.5029
Epoch 90/100
5238/5238 [==============================] - 2s 384us/step - loss: 8.0129 - acc: 0.5029
Epoch 91/100
5238/5238 [==============================] - 2s 391us/step - loss: 8.0129 - acc: 0.5029
Epoch 92/100
5238/5238 [==============================] - 2s 408us/step - loss: 8.0129 - acc: 0.5029
Epoch 93/100
5238/5238 [==============================] - 2s 405us/step - loss: 8.0129 - acc: 0.5029
Epoch 94/100
5238/5238 [==============================] - 2s 415us/step - loss: 8.0129 - acc: 0.5029
Epo

5266/5266 [==============================] - 2s 452us/step - loss: 8.0560 - acc: 0.5002
Epoch 63/100
5266/5266 [==============================] - 2s 447us/step - loss: 8.0560 - acc: 0.5002
Epoch 64/100
5266/5266 [==============================] - 2s 417us/step - loss: 8.0560 - acc: 0.5002
Epoch 65/100
5266/5266 [==============================] - 2s 431us/step - loss: 8.0560 - acc: 0.5002
Epoch 66/100
5266/5266 [==============================] - 2s 427us/step - loss: 8.0560 - acc: 0.5002
Epoch 67/100
5266/5266 [==============================] - 2s 437us/step - loss: 8.0560 - acc: 0.5002
Epoch 68/100
5266/5266 [==============================] - 2s 437us/step - loss: 8.0560 - acc: 0.5002
Epoch 69/100
5266/5266 [==============================] - 2s 439us/step - loss: 8.0560 - acc: 0.5002
Epoch 70/100
5266/5266 [==============================] - 2s 429us/step - loss: 8.0560 - acc: 0.5002
Epoch 71/100
5266/5266 [==============================] - 2s 437us/step - loss: 8.0560 - acc: 0.5002
Epo

5266/5266 [==============================] - 2s 388us/step - loss: 0.0137 - acc: 0.9968
Epoch 40/100
5266/5266 [==============================] - 2s 382us/step - loss: 5.1674e-04 - acc: 1.0000
Epoch 41/100
5266/5266 [==============================] - 2s 384us/step - loss: 3.3438e-04 - acc: 1.0000
Epoch 42/100
5266/5266 [==============================] - 2s 407us/step - loss: 1.1537e-04 - acc: 1.0000
Epoch 43/100
5266/5266 [==============================] - 2s 409us/step - loss: 8.3883e-05 - acc: 1.0000
Epoch 44/100
5266/5266 [==============================] - 2s 400us/step - loss: 5.7085e-05 - acc: 1.0000
Epoch 45/100
5266/5266 [==============================] - 2s 403us/step - loss: 5.3403e-05 - acc: 1.0000
Epoch 46/100
5266/5266 [==============================] - 2s 397us/step - loss: 3.9280e-05 - acc: 1.0000
Epoch 47/100
5266/5266 [==============================] - 2s 395us/step - loss: 3.5154e-05 - acc: 1.0000
Epoch 48/100
5266/5266 [==============================] - 2s 385us/step 

5266/5266 [==============================] - 2s 404us/step - loss: 0.0051 - acc: 0.9989
Epoch 16/100
5266/5266 [==============================] - 2s 405us/step - loss: 6.6084e-04 - acc: 1.0000
Epoch 17/100
5266/5266 [==============================] - 2s 398us/step - loss: 4.1791e-04 - acc: 1.0000
Epoch 18/100
5266/5266 [==============================] - 2s 410us/step - loss: 2.4846e-04 - acc: 1.0000
Epoch 19/100
5266/5266 [==============================] - 2s 414us/step - loss: 2.0199e-04 - acc: 1.0000
Epoch 20/100
5266/5266 [==============================] - 2s 391us/step - loss: 2.1587e-04 - acc: 1.0000
Epoch 21/100
5266/5266 [==============================] - 2s 408us/step - loss: 1.3624e-04 - acc: 1.0000
Epoch 22/100
5266/5266 [==============================] - 2s 397us/step - loss: 1.0590e-04 - acc: 1.0000
Epoch 23/100
5266/5266 [==============================] - 2s 412us/step - loss: 8.9589e-05 - acc: 1.0000
Epoch 24/100
5266/5266 [==============================] - 2s 396us/step 

5266/5266 [==============================] - 2s 389us/step - loss: 1.2202e-04 - acc: 1.0000
Epoch 94/100
5266/5266 [==============================] - 2s 391us/step - loss: 1.0108e-04 - acc: 1.0000
Epoch 95/100
5266/5266 [==============================] - 2s 398us/step - loss: 8.8415e-05 - acc: 1.0000
Epoch 96/100
5266/5266 [==============================] - 2s 372us/step - loss: 6.4566e-05 - acc: 1.0000
Epoch 97/100
5266/5266 [==============================] - 2s 398us/step - loss: 5.1364e-05 - acc: 1.0000
Epoch 98/100
5266/5266 [==============================] - 2s 393us/step - loss: 4.4330e-05 - acc: 1.0000
Epoch 99/100
5266/5266 [==============================] - 2s 395us/step - loss: 3.7687e-05 - acc: 1.0000
Epoch 100/100
1411/1411 [==============================] - 2s 1ms/step
[1317.   94.]
156/156 [==============================] - 0s 115us/step
[146.  10.]


In [10]:
noise = np.random.normal(loc=0,scale=1e-2,size=X_train.shape)
output = X_train + noise
X_train_all = np.concatenate((X_train, output), axis=0)
print(noise)
print(noise.shape)
print(output.shape)
print(X_train.shape)
print(X_train_all.shape)
for i in range(len(X_train)):
    if (X_train[i]==X_train_all[i]).all()==False:
        print(i)
for i in range(len(X_train)):
    ind = len(X_train)+i
    if (output[i]==X_train_all[ind]).all()==False:
        print(ind)
        

[[-0.00832716 -0.0016352   0.00077363 ...  0.00183968 -0.00465979
   0.00829321]
 [ 0.00584414 -0.00588995  0.00421623 ...  0.00297677 -0.01924222
  -0.00400722]
 [-0.01119581  0.00121366 -0.00490499 ...  0.01273558  0.01731577
   0.00500938]
 ...
 [ 0.00864477  0.00528896 -0.01210766 ...  0.01712098  0.00433754
  -0.0013977 ]
 [ 0.00705271 -0.00122142  0.00575762 ... -0.01171972  0.00219366
   0.01711726]
 [-0.00967948  0.00163952 -0.00307036 ...  0.01028011 -0.01576009
  -0.01723864]]
(1411, 474)
(1411, 474)
(1411, 474)
(2822, 474)


In [11]:
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_TP_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_FP_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_TN_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_FN_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
            for iRun in range(nRun):
                print(all_TP_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_FP_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_TN_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_FN_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])

for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_count_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_count_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])


[1316.   94.]
[0. 0.]
[  94. 1316.]
[0. 0.]
[1316.   94.]
[0. 0.]
[  94. 1316.]
[0. 0.]
[1313.   93.]
[1. 3.]
[  93. 1313.]
[3. 1.]
[1317.   93.]
[0. 0.]
[  93. 1317.]
[0. 0.]
[1317.   93.]
[0. 0.]
[  93. 1317.]
[0. 0.]
[1317.   93.]
[0. 0.]
[  93. 1317.]
[0. 0.]
[1317.   93.]
[0. 0.]
[  93. 1317.]
[0. 0.]
[1317.    6.]
[88.  0.]
[   6. 1317.]
[ 0. 88.]
[1317.   94.]
[0. 0.]
[  94. 1317.]
[0. 0.]
[1317.   94.]
[0. 0.]
[  94. 1317.]
[0. 0.]
[139.   1.]
[9. 8.]
[  1. 139.]
[8. 9.]
[142.   0.]
[10.  5.]
[  0. 142.]
[ 5. 10.]
[141.   1.]
[9. 6.]
[  1. 141.]
[6. 9.]
[141.   2.]
[9. 5.]
[  2. 141.]
[5. 9.]
[141.   2.]
[9. 5.]
[  2. 141.]
[5. 9.]
[139.   0.]
[11.  7.]
[  0. 139.]
[ 7. 11.]
[139.   1.]
[10.  7.]
[  1. 139.]
[ 7. 10.]
[145.   0.]
[10.  1.]
[  0. 145.]
[ 1. 10.]
[134.   2.]
[ 8. 12.]
[  2. 134.]
[12.  8.]
[140.   4.]
[6. 6.]
[  4. 140.]
[6. 6.]
[1316.   94.]
[0. 0.]
[  94. 1316.]
[0. 0.]
[1316.   94.]
[0. 0.]
[  94. 1316.]
[0. 0.]
[1316.   94.]
[0. 0.]
[  94. 1316.]
[0. 0.]
[131

In [12]:
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_time_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])


50.5678827
46.3822337
47.1511275
53.0018058
50.94166470000002
45.295047599999975
49.78580049999999
48.58641560000001
50.85853789999999
45.87759790000001
46.55111590000001
50.05037730000004
45.217466299999955
34.15604159999998
33.96559179999997
35.21884970000008
34.38546429999997
39.45160120000003
37.23201849999998
38.06074000000001
73.23924969999996
77.69205950000014
83.86184359999993
71.73928739999997
86.78888290000009
90.70137320000003
90.57380779999994
87.19688909999991
90.89302899999984
96.63720009999997
83.88643189999993
91.58415830000013
86.83466540000018
93.15142939999987
95.74925640000038
94.6649480000001
95.6176959999998
100.3900577000004
97.07613999999967
100.09920049999982
110.47443470000007
104.03958360000024
112.07344079999984
113.19626040000003
106.68365270000004
117.12757839999995
105.95673739999984
116.42578870000034
115.98688270000002
112.18846350000013
113.78436099999999
109.86275089999981
108.08744439999919
111.35109850000026
109.34805400000005
106.35040240000035
109

In [13]:
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_time_predict_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])


0.12159240000000437
0.1475307999999984
0.16043450000000803
0.1869914000000108
0.18579080000000658
0.2345532999999591
0.22750889999997526
0.24470199999996112
0.2913298999999938
0.2963346999999885
0.29683890000001156
0.3417527000000291
0.2857154000000719
0.2904850999999553
0.2965985999999248
0.3509693999999399
0.353150899999946
0.3842571999999791
0.3923553999999285
0.4283546000000342
0.5081410000000233
0.4950700000001689
0.5598404999998365
0.5259300000000167
0.6126322000000073
0.629740399999946
0.6940920000001825
0.7522243999999318
0.7009189000000333
0.76481200000012
0.7494891999999709
0.7911587000000964
0.8248414999998204
0.882664400000067
0.8712264000000687
0.9295038000000204
0.9191953999998077
0.9555438000002141
1.0032031999999163
1.0684732999998232
1.0055743000002622
1.062726399999974
1.0925156000002971
1.1252481999999873
1.1313341000000037
1.1441580999999132
1.166452500000105
1.210769999999684
1.2418649000001096
1.243440499999906
1.2760824999995748
1.300552400000015
1.34752269999989

In [14]:
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_time_predict_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])


0.01149570000000466
0.011887200000003872
0.012940600000007407
0.010862599999995837
0.010267499999997654
0.011393699999985074
0.011578100000008362
0.011808100000052946
0.011936600000012731
0.012959799999975985
0.01203269999996337
0.014415900000017245
0.009432699999933902
0.009316200000057506
0.009481800000003204
0.011343800000076953
0.010509200000001329
0.010635499999921194
0.009945199999947363
0.010749000000032538
0.015550399999938236
0.011756500000046799
0.01162690000001021
0.01237330000003567
0.014614600000186329
0.013450400000010632
0.012559199999941484
0.014246399999819914
0.015214499999956388
0.013647499999933643
0.01292650000004869
0.015047000000095068
0.013370099999974627
0.01698820000001433
0.012802299999748357
0.01387139999997089
0.013466500000049564
0.015367699999842444
0.01629929999990054
0.020104299999729847
0.013924800000040705
0.013477299999976822
0.014303500000096392
0.015454899999895133
0.015313100000184932
0.02024249999976746
0.014391900000191526
0.017786899999919115
0

In [15]:
# # restore
#model = tf.keras.models.load_model('my_resnet_model.h5')

# # test
#img_path = "../my_nn/dataset/test/medicine/IMG_20190717_135408_BURST91.jpg"
#img_path = "../my_nn/dataset/test/glue/IMG_20190717_135425_BURST91.jpg"

#img = image.load_img(img_path, target_size=(224, 224))

#plt.imshow(img)
#img = image.img_to_array(img)/ 255.0
#img = np.expand_dims(img, axis=0)  # 为batch添加第四维

#print(model.predict(img))
#np.argmax(model.predict(img))

#save_path =("./tmp_TP_EP%d") % (1)
#fl=open(save_path, 'w')
#lists=[line+"\n" for line in all_TP_train]
#fl.writelines(lists)